# **카카오 Geocode**

In [1]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [4]:
# 카카오 Rest API 키 등록
KAKAO_API_KEY = '4254bbc85c09f3ff26372511d2bd5d10' 

### **📍 좌표, 주소 반환**

**KaKao 지도/로컬 API Guideline**
- https://developers.kakao.com/product/map
- (로컬 >) **주소 검색**: 주소를 지도 위에 정확하게 표시할 수 있도록 좌표 정보를 제공합니다.
- (로컬 >) **좌표-행정구역정보 변환**: 다양한 좌표계의 좌표 값을 받아 해당 행정동, 법정동 정보를 제공합니다.

In [10]:
# 카카오 서버에 요청 후, 응답받은 데이터를 반환하는 함수
def json_request(url):
    headers = {'Authorization' : 'KakaoAK {}'.format(KAKAO_API_KEY)}
    res = requests.get(url, headers=headers)
    return res.text

#### **🔸 좌표 반환**
- 주소를 검색하면 좌표를 제공

In [18]:
# 주소 검색 (좌표 정보 제공)
def addr_lat_lon(addr):
    # 주소 -> 좌표 변환 요청 주소
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    
    try :
        res_json = json_request(url)
        res = json.loads(res_json)
        match_adr = res['documents'][0]['address']
    except :
        return 'NaN', 'NaN'

    return float(match_adr['x']), float(match_adr['y'])

In [19]:
addr_lat_lon('서울특별시 중구 세종대로 110')

(126.977829174031, 37.5663174209601)

#### **🔸 주소 반환**
- 좌표를 입력하면 주소 정보를 제공

In [46]:
# 좌표-행정구역정보 변환
def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    # 좌표 -> 주소 변환 api 요청 주소

    # 위경도에 매칭되는 '최종 번지'까지 반환
    URL = 'https://dapi.kakao.com/v2/local/geo/coord2address.json'
    # 위경도를 포함하는 '행정구역'까지 반환
    # URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
    
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)

    try :
        json_res = json_request(url=url)
        res = json.loads(json_res)
        doc = res['documents'][0]
        adr1 = doc['road_address']['address_name'] # 도로 주소
        adr2 = doc['address']['address_name'] # 지번 주소
    except :
        return '주소 검색 실패'
    return adr1, adr2

In [29]:
lat = 37.5663174209601
lng = 126.977829174031

In [41]:
# reverse_geocode(lng, lat)['documents'][0].keys()
# reverse_geocode(lng, lat)['documents'][0].keys() => dict_keys(['road_address', 'address'])
print('도로 주소: ', reverse_geocode(lng, lat)['documents'][0]['road_address']['address_name'])
print('지번 주소: ', reverse_geocode(lng, lat)['documents'][0]['address']['address_name'])

도로 주소:  서울특별시 중구 세종대로 110
지번 주소:  서울 중구 태평로1가 31


In [47]:
reverse_geocode(lng, lat)

('서울특별시 중구 세종대로 110', '서울 중구 태평로1가 31')